In [7]:
import os
import pandas as pd

from sklearn.model_selection import KFold
from sklearn.metrics import roc_auc_score, accuracy_score, f1_score, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import MinMaxScaler
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from tabulate import tabulate
from sklearn.model_selection import cross_val_score


**Load The Data**

In [8]:
current_dir = os.getcwd() 
relative_path_train = os.path.join('..', 'data', 'preprocessed_train_data.csv')
relative_path_test = os.path.join('..', 'data', 'preprocessed_test_data.csv')

preprocessed_train_data = pd.read_csv(os.path.join(current_dir, relative_path_train))
preprocessed_test_data = pd.read_csv(os.path.join(current_dir, relative_path_test))

**Data Spliting**

In [9]:
x_train = preprocessed_train_data.drop(["satisfaction"], axis = 1)
y_train = preprocessed_train_data["satisfaction"]

x_test = preprocessed_test_data.drop(["satisfaction"], axis = 1)
y_test = preprocessed_test_data["satisfaction"]

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

**Create a list of classifiers**

In [10]:
classifiers = [
    ("Logistic Regression", LogisticRegression()),
    ("Decision Tree", DecisionTreeClassifier()),
    ("Random Forest", RandomForestClassifier()),
    ("Gradient Boosting", GradientBoostingClassifier()),
    ("K-Nearest Neighbors", KNeighborsClassifier()),
    ("Gaussian Naive Bayes", GaussianNB()),
    ("Multi-layer Perceptron", MLPClassifier()),
    ("XGBoost", XGBClassifier()),
    ("CatBoost", CatBoostClassifier()), 
    ("AdaBoost", AdaBoostClassifier())
]

In [11]:
# Initialize an empty DataFrame to store the results
results_df = pd.DataFrame(columns=["Classifier", "Balanced Accuracy", "Training Accuracy", "Validation Accuracy", "Testing Accuracy", "F1 Score", "Precision", "Recall"])


**Train and evaluate each classifier**

In [12]:
for name, clf in classifiers:
    print(name)
    
    # Fit the classifier on the entire training set
    clf.fit(x_train, y_train)
    
    # Calculate training accuracy
    accuracy_train = accuracy_score(y_train, clf.predict(x_train))
    
    # Calculate validation accuracy using k-fold cross-validation
    accuracy_validation = cross_val_score(clf, x_train, y_train, cv=k_fold, scoring='accuracy').mean()
    
    y_prob = clf.predict_proba(x_test)[:, 1]
    auc_score = roc_auc_score(y_test, y_prob)
    # Make predictions on the test set
    y_pred = clf.predict(x_test)
    f1 = f1_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    # Create a temporary Series with results for this classifier
    new_row = pd.Series({
        "Classifier": name,
        "Balanced Accuracy": auc_score,
        "Training Accuracy": accuracy_train,
        "Validation Accuracy": accuracy_validation,
        "Testing Accuracy": accuracy_score(y_test, y_pred),
        "F1 Score": f1,
        "Precision": precision,
        "Recall": recall
    })

    # Append the Series as a new row to the results DataFrame
    results_df = pd.concat([results_df, new_row.to_frame().T], ignore_index=True)

# Sort the DataFrame by Balanced Accuracy in descending order
results_df = results_df.sort_values(by="Balanced Accuracy", ascending=False)

Logistic Regression
Decision Tree
Random Forest
Gradient Boosting
K-Nearest Neighbors
Gaussian Naive Bayes
Multi-layer Perceptron


/home/donia/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/donia/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/donia/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/donia/.local/lib/python3.8/site-packages/sklearn/neural_network/_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
/home/donia/.local/lib/python3.8/site-packages/sklearn/n

XGBoost
CatBoost
Learning rate set to 0.074823
0:	learn: 0.5925347	total: 215ms	remaining: 3m 34s
1:	learn: 0.4853586	total: 243ms	remaining: 2m 1s
2:	learn: 0.4290905	total: 267ms	remaining: 1m 28s
3:	learn: 0.3833252	total: 292ms	remaining: 1m 12s
4:	learn: 0.3513727	total: 316ms	remaining: 1m 2s
5:	learn: 0.3089257	total: 343ms	remaining: 56.8s
6:	learn: 0.2883640	total: 370ms	remaining: 52.5s
7:	learn: 0.2711136	total: 396ms	remaining: 49.1s
8:	learn: 0.2577476	total: 426ms	remaining: 46.9s
9:	learn: 0.2402199	total: 461ms	remaining: 45.7s
10:	learn: 0.2314451	total: 486ms	remaining: 43.7s
11:	learn: 0.2185457	total: 510ms	remaining: 42s
12:	learn: 0.2103959	total: 535ms	remaining: 40.6s
13:	learn: 0.2041481	total: 561ms	remaining: 39.5s
14:	learn: 0.1956322	total: 586ms	remaining: 38.5s
15:	learn: 0.1903867	total: 611ms	remaining: 37.6s
16:	learn: 0.1835245	total: 639ms	remaining: 36.9s
17:	learn: 0.1753302	total: 671ms	remaining: 36.6s
18:	learn: 0.1715682	total: 696ms	remaining:

In [13]:
# Print the results table with styling
styled_results = results_df.style.background_gradient(cmap='Blues', subset=["Balanced Accuracy", "Training Accuracy", "Validation Accuracy", "Testing Accuracy", "F1 Score", "Precision", "Recall"])

In [14]:
styled_results

,Classifier,Balanced Accuracy,Training Accuracy,Validation Accuracy,Testing Accuracy,F1 Score,Precision,Recall
8,CatBoost,0.994832,0.973658,0.963707,0.962786,0.956157,0.972635,0.940229
6,Multi-layer Perceptron,0.993326,0.967999,0.959010,0.956742,0.949448,0.957806,0.941236
2,Random Forest,0.993260,0.999990,0.962523,0.961917,0.955104,0.972220,0.938581
3,Gradient Boosting,0.987055,0.940878,0.940849,0.941216,0.930771,0.946447,0.915606
9,AdaBoost,0.975945,0.925566,0.925162,0.924347,0.911297,0.922449,0.900412
7,XGBoost,0.973716,0.972744,0.962716,0.867025,0.821432,0.976909,0.708650
4,K-Nearest Neighbors,0.972847,0.953139,0.932784,0.932367,0.919161,0.949283,0.890892
1,Decision Tree,0.941725,1.000000,0.944247,0.942757,0.933718,0.933248,0.934188
0,Logistic Regression,0.923548,0.874192,0.874230,0.871568,0.847978,0.866826,0.829931
5,Gaussian Naive Bayes,0.913202,0.849515,0.849544,0.845060,0.817768,0.830424,0.805492


In [10]:
'''
CatBoost achieved the highest accuracy, F1 Score & Balanced Accuracy
'''

'\nCatBoost achieved the highest accuracy, F1 Score & Balanced Accuracy\n'

**MultinomialNB Naive Bayes**

scale train and test data

In [40]:
# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

Train the MultinomialNB classifier

In [41]:
# Create an instance of the classifier
clf = MultinomialNB()
clf.fit(x_train_scaled, y_train)
y_pred = clf.predict(x_test_scaled)
y_prob = clf.predict_proba(x_test_scaled)[:, 1]
auc_score = roc_auc_score(y_test, y_prob)
accuracy_train = accuracy_score(y_train, clf.predict(x_train_scaled))
accuracy_test = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

accuracy_validation = cross_val_score(clf, x_train_scaled, y_train, cv=k_fold, scoring='accuracy').mean()
    
print("Balanced Accuracy: ", auc_score)
print("Training Accuracy: ", accuracy_train)
print("Testing Accuracy: ", accuracy_test)
print("Validation Accuracy: ", accuracy_validation)
print("F1 Score: ", f1)
print("Precision: ", precision)
print("Recall: ", recall)

Balanced Accuracy:  0.8741036230929793
Training Accuracy:  0.7680455035417308
Testing Accuracy:  0.7649034093153716
Validation Accuracy:  0.7680455332217699
F1 Score:  0.7330791657322269
Precision:  0.7187335092348285
Recall:  0.7480091533180778


**MultinomialNB Naive Bayes with applying grouping on columns with continous values**

Load the data

In [3]:
current_dir = os.getcwd() 
relative_path_train = os.path.join('..', 'data', 'preprocessed_train_data_after_grouping.csv')
relative_path_test = os.path.join('..', 'data', 'preprocessed_test_data_after_grouping.csv')

new_preprocessed_train_data = pd.read_csv(os.path.join(current_dir, relative_path_train))
new_preprocessed_test_data = pd.read_csv(os.path.join(current_dir, relative_path_test))

In [4]:
x_train = new_preprocessed_train_data.drop(["satisfaction"], axis = 1)
y_train = new_preprocessed_train_data["satisfaction"]

x_test = new_preprocessed_test_data.drop(["satisfaction"], axis = 1)
y_test = new_preprocessed_test_data["satisfaction"]

scale train and test data

In [5]:
# Create an instance of MinMaxScaler
scaler = MinMaxScaler()

# Fit the scaler on the training data and transform both training and test data
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test)

k_fold = KFold(n_splits=10, shuffle=True, random_state=42)

Train the MultinomialNB classifier

In [6]:
# Create an instance of the classifier
clf = MultinomialNB()
clf.fit(x_train_scaled, y_train)
y_pred = clf.predict(x_test_scaled)
y_prob = clf.predict_proba(x_test_scaled)[:, 1]
auc_score = roc_auc_score(y_test, y_prob)
accuracy_train = accuracy_score(y_train, clf.predict(x_train_scaled))
accuracy_test = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)

accuracy_validation = cross_val_score(clf, x_train_scaled, y_train, cv=k_fold, scoring='accuracy').mean()
    
print("Balanced Accuracy: ", auc_score)
print("Training Accuracy: ", accuracy_train)
print("Testing Accuracy: ", accuracy_test)
print("Validation Accuracy: ", accuracy_validation)
print("F1 Score: ", f1)
print("Precision: ", precision)
print("Recall: ", recall)

Balanced Accuracy:  0.8660401815904305
Training Accuracy:  0.7687865722205113
Testing Accuracy:  0.7656145063801209
Validation Accuracy:  0.7687288538824919
F1 Score:  0.7336236699142458
Precision:  0.7199506520972858
Recall:  0.7478260869565218
